In [3]:
import pandas as pd
import numpy as np
# Machine learning components
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import precision_recall_curve
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    average_precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report,
    roc_auc_score,
    roc_curve
)
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

In [5]:
data = pd.read_csv("../raw_data/fraudTrain.csv")

y = data["is_fraud"]
X = data.drop(columns=["is_fraud"])


In [10]:
# Drop useless / leakage-prone columns
X.drop(
    columns=[
        'Unnamed: 0','unix_time','cc_num','first','last',
        'street','city','state','zip','city_pop',
        'trans_num','job','merchant'
    ],
    inplace=True,
    errors="ignore"
)

if 'dob' in X.columns:
    X['dob'] = pd.to_datetime(X['dob'])

if 'trans_date_trans_time' in X.columns:
    X['trans_date_trans_time'] = pd.to_datetime(X['trans_date_trans_time'])

if {'dob','trans_date_trans_time'}.issubset(X.columns):
    X['age'] = (X['trans_date_trans_time'] - X['dob']).dt.days / 365.25

if 'trans_date_trans_time' in X.columns:
    X['trans_month'] = X['trans_date_trans_time'].dt.month
    X['trans_day']   = X['trans_date_trans_time'].dt.day
    X['trans_hour']  = X['trans_date_trans_time'].dt.hour

if 'trans_month' in X.columns:
    X['trans_month_sin'] = np.sin(2 * np.pi * X['trans_month'] / 12)
    X['trans_month_cos'] = np.cos(2 * np.pi * X['trans_month'] / 12)

if 'trans_day' in X.columns:
    X['trans_day_sin'] = np.sin(2 * np.pi * X['trans_day'] / 31)
    X['trans_day_cos'] = np.cos(2 * np.pi * X['trans_day'] / 31)

if 'trans_hour' in X.columns:
    X['trans_hour_sin'] = np.sin(2 * np.pi * X['trans_hour'] / 24)
    X['trans_hour_cos'] = np.cos(2 * np.pi * X['trans_hour'] / 24)

X.drop(
    columns=[
        'dob','trans_date_trans_time',
        'trans_month','trans_day','trans_hour'
    ],
    inplace=True,
    errors="ignore"
)


In [11]:
# Train vs (Val + Test)
X_train, X_temp, y_train, y_temp = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

# Validation vs Test
X_val, X_test, y_val, y_test = train_test_split(
    X_temp,
    y_temp,
    test_size=0.5,
    random_state=42,
    stratify=y_temp
)

print("Train:", X_train.shape)
print("Val:  ", X_val.shape)
print("Test: ", X_test.shape)

Train: (907672, 14)
Val:   (194501, 14)
Test:  (194502, 14)


In [12]:
#Part 2 (Scaling) We apply it to only two columns here. Fit and transform on training set. Transform on test set. See how to use Ashenafi functions.
# amt_scaler = RobustScaler().set_output(transform="pandas")
# age_scaler = RobustScaler().set_output(transform="pandas")

# # robust_scaler = RobustScaler().set_output(transform='pandas')
# X_train['amt'] = amt_scaler.fit_transform(X_train[['amt']])
# X_val['amt']   = amt_scaler.transform(X_val[['amt']])
# X_test['amt']  = amt_scaler.transform(X_test[['amt']])

# X_train['age'] = age_scaler.fit_transform(X_train[['age']])
# X_val['age']   = age_scaler.transform(X_val[['age']])
# X_test['age']  = age_scaler.transform(X_test[['age']])

numeric_features = X_train.select_dtypes(include="number").columns.tolist()
categorical_features = X_train.select_dtypes(include="object").columns.tolist()

neg = (y_train == 0).sum()
pos = (y_train == 1).sum()
scale_pos_weight = neg / pos

preprocessor = ColumnTransformer(
    transformers=[
        ("num", RobustScaler(), numeric_features),
        ("cat", OneHotEncoder(
            handle_unknown="ignore",
            drop="if_binary"
        ), categorical_features)
    ],
    remainder="drop"
)

pipeline = Pipeline([
    ("prep", preprocessor),
    ("model", xgb.XGBClassifier(
        n_estimators=400,
        max_depth=6,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        scale_pos_weight=scale_pos_weight,
        eval_metric="logloss",
        tree_method="hist",
        random_state=42,
        n_jobs=-1,
        use_label_encoder=False
    ))
])

In [13]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

cv_scores = cross_val_score(
    pipeline,
    X_train,
    y_train,
    scoring="average_precision",
    cv=cv,
    n_jobs=-1
)

print("CV PR-AUC scores:", cv_scores)
print("Mean CV PR-AUC:", cv_scores.mean())

CV PR-AUC scores: [nan nan nan nan nan]
Mean CV PR-AUC: nan


In [98]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num', RobustScaler(),
                                                  ['amt', 'lat', 'long',
                                                   'merch_lat', 'merch_long',
                                                   'age', 'trans_month_sin',
                                                   'trans_month_cos',
                                                   'trans_day_sin',
                                                   'trans_day_cos',
                                                   'trans_hour_sin',
                                                   'trans_hour_cos']),
                                                 ('cat',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore'),
                                                  ['category', 'gender'])])),
                ('model',
                 XGBClassifier(base_...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.1,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=6, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=400, n_jobs=-1,
                               num_parallel_tree=None, random_state=42, ...))])

In [14]:
pipeline.fit(X_train, y_train)

y_val_proba = pipeline.predict_proba(X_val)[:, 1]

thresholds = np.linspace(0.01, 0.3, 100)
f1_scores = [f1_score(y_val, y_val_proba >= t) for t in thresholds]

best_threshold = thresholds[np.argmax(f1_scores)]

print("Best validation threshold:", best_threshold)


Best validation threshold: 0.3


In [ ]:
y_test_proba = pipeline.predict_proba(X_test)[:, 1]
y_test_pred = (y_test_proba >= best_threshold).astype(int)

print("TEST PR-AUC:", average_precision_score(y_test, y_test_proba))
print("Accuracy:", accuracy_score(y_test, y_test_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_test_pred))
print("\nClassification Report:\n", classification_report(y_test, y_test_pred))


TEST PR-AUC: 0.9537342720620718
Accuracy: 0.9935990375420304

Confusion Matrix:
 [[192164   1212]
 [    33   1093]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.99      1.00    193376
           1       0.47      0.97      0.64      1126

    accuracy                           0.99    194502
   macro avg       0.74      0.98      0.82    194502
weighted avg       1.00      0.99      0.99    194502



#### fit the model

In [18]:
# fraudTest.csv

external_test = pd.read_csv("../raw_data/fraudtest.csv")

y_test_ext = external_test["is_fraud"]
X_test_ext = external_test.drop(columns=["is_fraud"])

# Apply SAME feature engineering manually
X_test_ext = X_test_ext.copy()

X_test_ext.drop(
    columns=[
        'Unnamed: 0','unix_time','cc_num','first','last',
        'street','city','state','zip','city_pop',
        'trans_num','job','merchant'
    ],
    inplace=True,
    errors="ignore"
)

X_test_ext['dob'] = pd.to_datetime(X_test_ext['dob'])
X_test_ext['trans_date_trans_time'] = pd.to_datetime(X_test_ext['trans_date_trans_time'])

X_test_ext['age'] = (
    X_test_ext['trans_date_trans_time'] - X_test_ext['dob']
).dt.days / 365.25

X_test_ext['trans_month'] = X_test_ext['trans_date_trans_time'].dt.month
X_test_ext['trans_day']   = X_test_ext['trans_date_trans_time'].dt.day
X_test_ext['trans_hour']  = X_test_ext['trans_date_trans_time'].dt.hour

X_test_ext['trans_month_sin'] = np.sin(2 * np.pi * X_test_ext['trans_month'] / 12)
X_test_ext['trans_month_cos'] = np.cos(2 * np.pi * X_test_ext['trans_month'] / 12)

X_test_ext['trans_day_sin'] = np.sin(2 * np.pi * X_test_ext['trans_day'] / 31)
X_test_ext['trans_day_cos'] = np.cos(2 * np.pi * X_test_ext['trans_day'] / 31)

X_test_ext['trans_hour_sin'] = np.sin(2 * np.pi * X_test_ext['trans_hour'] / 24)
X_test_ext['trans_hour_cos'] = np.cos(2 * np.pi * X_test_ext['trans_hour'] / 24)

X_test_ext.drop(
    columns=['dob','trans_date_trans_time','trans_month','trans_day','trans_hour'],
    inplace=True
)

y_test_ext_proba = pipeline.predict_proba(X_test_ext)[:, 1]
y_test_ext_pred = (y_test_ext_proba >= best_threshold).astype(int)

print("EXTERNAL TEST PR-AUC:",
      average_precision_score(y_test_ext, y_test_ext_proba))


EXTERNAL TEST PR-AUC: 0.8265041510926379


In [ ]:
# prediction using only the test dataset

In [19]:
print(classification_report(y_test_ext, y_test_ext_pred))
print(confusion_matrix(y_test_ext, y_test_ext_pred))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00    553574
           1       0.38      0.89      0.54      2145

    accuracy                           0.99    555719
   macro avg       0.69      0.94      0.77    555719
weighted avg       1.00      0.99      1.00    555719

[[550518   3056]
 [   244   1901]]
